In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('CPU is available')

GPU is available


In [2]:
import sys
sys.path.append('../..')

from reservoirgrid.models import Reservoir
from reservoirgrid.helpers import viz

In [3]:
# 'your_file.csv' with the path to your CSV file
import pandas as pd

features = ['Open', 'High', 'Low', 'Close', 'VWAP', 'Volume']
df = pd.read_csv('../../reservoirgrid/datasets/ADANIPORTS.csv')

In [4]:
df

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123,0.2741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,2021-04-26,ADANIPORTS,EQ,725.35,733.00,739.65,728.90,729.2,730.75,733.25,9390549,6.885658e+14,116457.0,838079,0.0892
3318,2021-04-27,ADANIPORTS,EQ,730.75,735.00,757.50,727.35,748.6,749.15,747.67,20573107,1.538191e+15,236896.0,1779639,0.0865
3319,2021-04-28,ADANIPORTS,EQ,749.15,755.00,760.00,741.10,743.4,746.25,751.02,11156977,8.379106e+14,130847.0,1342353,0.1203
3320,2021-04-29,ADANIPORTS,EQ,746.25,753.20,765.85,743.40,746.4,746.75,753.06,13851910,1.043139e+15,153293.0,1304895,0.0942


In [5]:
import numpy as np
 
df[features] = df[features].astype(float)
Dataset = df[features].values

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(Dataset, test_size=0.01, shuffle=False)
train.shape , test.shape

((3288, 6), (34, 6))

In [7]:
train_inputs = train[:-1]
train_targets = train[1:]
train_inputs = torch.tensor(train_inputs, dtype=torch.float32).to(device)
train_targets = torch.tensor(train_targets, dtype=torch.float32).to(device)

In [8]:
train_inputs.shape, train_targets.shape

(torch.Size([3287, 6]), torch.Size([3287, 6]))

In [9]:
from reservoirgrid.models import Reservoir

In [29]:
Resnifty = Reservoir(
    input_dim=6,                
    reservoir_dim=1000,          
    output_dim=6,               
    spectral_radius=1.25,       
    leak_rate=0.5,              
    sparsity=0.5,               
    input_scaling=0.5           
)

Resnifty.to(device)


Reservoir(
  (readout): Linear(in_features=1000, out_features=6, bias=True)
)

In [30]:
Resnifty.train_readout(train_inputs, train_targets)
predictions = Resnifty.predict(train_inputs, steps=len(test[:,3]))
predictions_cpu = predictions.detach().cpu().numpy()

viz.compare_plot([predictions_cpu[:3], test[:,3]], titles=["Predictions", "True"])

Readout training complete.


In [31]:
predictions_cpu[:,3], test[:,3]

(array([274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159, 274.30666159, 274.30666159,
        274.30666159, 274.30666159]),
 array([750.75, 745.35, 728.15, 719.05, 717.35, 688.25, 679.65, 686.25,
        721.9 , 738.55, 706.85, 688.25, 703.05, 710.55, 702.4 , 736.25,
        742.25, 835.55, 814.75, 823.  , 823.6 , 744.65, 731.05, 750.35,
        755.5 , 720.95, 720.45, 737.8 , 725.35, 730.75, 749.15, 746.25,
        746.75, 730.05]))